In [ ]:
# using an XGBoost model, the goal is to predict lap times
# TRAINING DATA: all data ingested from the 2024 season

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Data Gathering

In [ ]:
# pull the lap times data from the lap_times_df.csv file
lap_data = pd.read_csv("../data/lap_times_df.csv")
X = lap_data[["race_lap", "tire_age", "stint", "track_temp", "air_temp", "compound", "track"]]  # NEED WEATHER DATA
y = lap_data["lap_times_sec"]

In [ ]:
# cover the categorical columns

## Model Building

## Model Optimization